In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor, plot_tree
from sklearn.tree import export_graphviz
import graphviz

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn import preprocessing

import gc
import xgboost as xgb
RANDOM_STATE=42

# Cargamos notificaciones

In [2]:
# Catgamos el dataset de train:
comu_file='Comunicaciones_train.csv'
comu=pd.read_csv(comu_file)
comu.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
comu

,id,Id_Producto,Tipo,Producto-Tipo,Tipo_comunicacion,Fecha,Periodo,Lectura
0,1,B,B,B-B,A,2020-03-02,202003,0
1,1,E,E,E-E,A,2020-03-20,202003,0
2,1,B,B,B-B,A,2020-01-09,202001,0
3,1,B,B,B-B,A,2020-01-28,202001,0
4,1,B,B,B-B,A,2020-01-21,202001,1
...,...,...,...,...,...,...,...,...
1358409,79539,B,B,B-B,A,2020-04-30,202004,0
1358410,79539,B,B,B-B,A,2020-06-11,202006,0
1358411,79539,C,D,C-D,A,2020-05-11,202005,0
1358412,79539,C,D,C-D,A,2020-07-20,202007,1


In [4]:
comu['Periodo']=pd.to_datetime(comu['Periodo'],format='%Y%m')

In [5]:
comu

,id,Id_Producto,Tipo,Producto-Tipo,Tipo_comunicacion,Fecha,Periodo,Lectura
0,1,B,B,B-B,A,2020-03-02,2020-03-01,0
1,1,E,E,E-E,A,2020-03-20,2020-03-01,0
2,1,B,B,B-B,A,2020-01-09,2020-01-01,0
3,1,B,B,B-B,A,2020-01-28,2020-01-01,0
4,1,B,B,B-B,A,2020-01-21,2020-01-01,1
...,...,...,...,...,...,...,...,...
1358409,79539,B,B,B-B,A,2020-04-30,2020-04-01,0
1358410,79539,B,B,B-B,A,2020-06-11,2020-06-01,0
1358411,79539,C,D,C-D,A,2020-05-11,2020-05-01,0
1358412,79539,C,D,C-D,A,2020-07-20,2020-07-01,1


In [7]:
comu.groupby(['id','Producto-Tipo']).size()

id     Producto-Tipo
1      B-B              20
       D-E               1
       E-E               1
2      B-B              16
       C-D               3
                        ..
79538  E-E               1
79539  B-B              11
       C-D               6
       D-E               1
       E-E               1
Length: 270093, dtype: int64

In [8]:
comu.groupby(['id','Periodo']).size()

id     Periodo   
1      2019-12-01    3
       2020-01-01    4
       2020-02-01    3
       2020-03-01    4
       2020-04-01    4
                    ..
79539  2020-03-01    2
       2020-04-01    3
       2020-05-01    1
       2020-06-01    1
       2020-07-01    1
Length: 667411, dtype: int64

In [9]:
comu.groupby(['id','Periodo','Producto-Tipo']).size()

id     Periodo     Producto-Tipo
1      2019-12-01  B-B              3
       2020-01-01  B-B              4
       2020-02-01  B-B              3
       2020-03-01  B-B              2
                   D-E              1
                                   ..
79539  2020-03-01  E-E              1
       2020-04-01  B-B              3
       2020-05-01  C-D              1
       2020-06-01  B-B              1
       2020-07-01  C-D              1
Length: 923590, dtype: int64

In [10]:
comu['Periodo'].value_counts(dropna=False)

2020-03-01    198974
2020-01-01    169450
2020-02-01    165638
2019-07-01    140606
2019-12-01    126371
2020-04-01    118828
2019-08-01    113702
2020-07-01     87586
2020-06-01     76157
2019-06-01     75109
2019-05-01     33592
2020-05-01     32083
2019-03-01     16952
2019-11-01      3365
2019-04-01         1
Name: Periodo, dtype: int64

['2019-04-01', 
'2019-07-01',
'2019-10-01', 
'2020-01-01',
'2020-04-01',
'2020-07-01',
'2020-10-01']

In [6]:
comu.loc[ (comu['Periodo']==pd.to_datetime('201902',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('201903',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('201904',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('201904',format='%Y%m')

In [7]:
comu['PeriodoT'].value_counts()

2019-04-01    16953
Name: PeriodoT, dtype: int64

In [8]:
comu.loc[ (comu['Periodo']==pd.to_datetime('201905',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('201906',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('201907',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('201907',format='%Y%m')

In [9]:
comu.loc[ (comu['Periodo']==pd.to_datetime('201908',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('201909',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('201910',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('201910',format='%Y%m')

In [10]:
comu.loc[ (comu['Periodo']==pd.to_datetime('201911',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('201912',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('202001',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202001',format='%Y%m')

In [11]:
comu.loc[ (comu['Periodo']==pd.to_datetime('202002',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('202003',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('202004',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202004',format='%Y%m')

In [12]:
comu.loc[ (comu['Periodo']==pd.to_datetime('202005',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('202006',format='%Y%m')) |
           (comu['Periodo']==pd.to_datetime('202007',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202007',format='%Y%m')

In [13]:
comu['PeriodoT'].value_counts()

2020-04-01    483440
2020-01-01    299186
2019-07-01    249307
2020-07-01    195826
2019-10-01    113702
2019-04-01     16953
Name: PeriodoT, dtype: int64

In [14]:
comu

,id,Id_Producto,Tipo,Producto-Tipo,Tipo_comunicacion,Fecha,Periodo,Lectura,PeriodoT
0,1,B,B,B-B,A,2020-03-02,2020-03-01,0,2020-04-01
1,1,E,E,E-E,A,2020-03-20,2020-03-01,0,2020-04-01
2,1,B,B,B-B,A,2020-01-09,2020-01-01,0,2020-01-01
3,1,B,B,B-B,A,2020-01-28,2020-01-01,0,2020-01-01
4,1,B,B,B-B,A,2020-01-21,2020-01-01,1,2020-01-01
...,...,...,...,...,...,...,...,...,...
1358409,79539,B,B,B-B,A,2020-04-30,2020-04-01,0,2020-04-01
1358410,79539,B,B,B-B,A,2020-06-11,2020-06-01,0,2020-07-01
1358411,79539,C,D,C-D,A,2020-05-11,2020-05-01,0,2020-07-01
1358412,79539,C,D,C-D,A,2020-07-20,2020-07-01,1,2020-07-01


In [15]:
comu.isnull().sum()

id                   0
Id_Producto          0
Tipo                 0
Producto-Tipo        0
Tipo_comunicacion    0
Fecha                0
Periodo              0
Lectura              0
PeriodoT             0
dtype: int64

In [46]:
comu.groupby(['id','PeriodoT','Producto-Tipo']).size().reset_index(drop=False, name='Count')

,id,PeriodoT,Producto-Tipo,Count
0,1,2020-01-01,B-B,7
1,1,2020-04-01,B-B,9
2,1,2020-04-01,D-E,1
3,1,2020-04-01,E-E,1
4,1,2020-07-01,B-B,4
...,...,...,...,...
627132,79539,2020-04-01,B-B,4
627133,79539,2020-04-01,D-E,1
627134,79539,2020-04-01,E-E,1
627135,79539,2020-07-01,B-B,1


In [47]:
comu

,id,Id_Producto,Tipo,Producto-Tipo,Tipo_comunicacion,Fecha,Periodo,Lectura,PeriodoT
0,1,B,B,B-B,A,2020-03-02,2020-03-01,0,2020-04-01
1,1,E,E,E-E,A,2020-03-20,2020-03-01,0,2020-04-01
2,1,B,B,B-B,A,2020-01-09,2020-01-01,0,2020-01-01
3,1,B,B,B-B,A,2020-01-28,2020-01-01,0,2020-01-01
4,1,B,B,B-B,A,2020-01-21,2020-01-01,1,2020-01-01
...,...,...,...,...,...,...,...,...,...
1358409,79539,B,B,B-B,A,2020-04-30,2020-04-01,0,2020-04-01
1358410,79539,B,B,B-B,A,2020-06-11,2020-06-01,0,2020-07-01
1358411,79539,C,D,C-D,A,2020-05-11,2020-05-01,0,2020-07-01
1358412,79539,C,D,C-D,A,2020-07-20,2020-07-01,1,2020-07-01


In [16]:
comu['Tipo_comunicacion'].value_counts(dropna=False)

A    1358414
Name: Tipo_comunicacion, dtype: int64

In [17]:
comu['Lectura'].value_counts(dropna=False)

0    910946
1    447468
Name: Lectura, dtype: int64

In [18]:
comu_=comu.pivot_table( index=['id','PeriodoT','Producto-Tipo'],
                         values=['Lectura'],
                         aggfunc=[len,np.sum,np.mean,np.max,np.min])

In [19]:
comu_.reset_index(drop=False,inplace=True)

In [20]:
comu_

,id,PeriodoT,Producto-Tipo,len,sum,mean,amax,amin
,,,,Lectura,Lectura,Lectura,Lectura,Lectura
0,1,2020-01-01,B-B,7,1,0.142857,1,0
1,1,2020-04-01,B-B,9,2,0.222222,1,0
2,1,2020-04-01,D-E,1,0,0.000000,0,0
3,1,2020-04-01,E-E,1,0,0.000000,0,0
4,1,2020-07-01,B-B,4,2,0.500000,1,0
...,...,...,...,...,...,...,...,...
627132,79539,2020-04-01,B-B,4,1,0.250000,1,0
627133,79539,2020-04-01,D-E,1,0,0.000000,0,0
627134,79539,2020-04-01,E-E,1,0,0.000000,0,0


In [21]:
comu_.columns=['id','PeriodoT','Producto-Tipo','Count-comu','Lectura','LecturaMean','LecturaMax','LecturaMin']

In [22]:
comu_

,id,PeriodoT,Producto-Tipo,Count-comu,Lectura,LecturaMean,LecturaMax,LecturaMin
0,1,2020-01-01,B-B,7,1,0.142857,1,0
1,1,2020-04-01,B-B,9,2,0.222222,1,0
2,1,2020-04-01,D-E,1,0,0.000000,0,0
3,1,2020-04-01,E-E,1,0,0.000000,0,0
4,1,2020-07-01,B-B,4,2,0.500000,1,0
...,...,...,...,...,...,...,...,...
627132,79539,2020-04-01,B-B,4,1,0.250000,1,0
627133,79539,2020-04-01,D-E,1,0,0.000000,0,0
627134,79539,2020-04-01,E-E,1,0,0.000000,0,0
627135,79539,2020-07-01,B-B,1,0,0.000000,0,0


# Cargamos comu test

In [23]:
# Catgamos el dataset de train:
comu_file='Comunicaciones_test.csv'
comu_test=pd.read_csv(comu_file)
comu_test.drop('Unnamed: 0',axis=1,inplace=True)

In [56]:
comu_test

,id,Id_Producto,Tipo,Producto-Tipo,Tipo_comunicacion,Fecha,Periodo,Lectura
0,1,B,B,B-B,A,2020-08-07,202008,1
1,1,B,B,B-B,A,2020-08-27,202008,1
2,1,B,B,B-B,A,2020-09-30,202009,0
3,1,B,B,B-B,A,2020-10-28,202010,1
4,1,B,B,B-B,A,2020-10-15,202010,0
...,...,...,...,...,...,...,...,...
230246,79537,B,B,B-B,A,2020-10-28,202010,0
230247,79538,B,B,B-B,A,2020-10-28,202010,1
230248,79539,C,D,C-D,A,2020-08-31,202008,0
230249,79539,C,D,C-D,A,2020-09-02,202009,0


In [24]:
comu_test['Periodo']=pd.to_datetime(comu_test['Periodo'],format='%Y%m')

In [25]:
comu_test['Periodo'].value_counts()

2020-10-01    126297
2020-08-01     65795
2020-09-01     38159
Name: Periodo, dtype: int64

In [26]:
comu_test.loc[ (comu_test['Periodo']==pd.to_datetime('202008',format='%Y%m')) |
           (comu_test['Periodo']==pd.to_datetime('202009',format='%Y%m')) |
           (comu_test['Periodo']==pd.to_datetime('202010',format='%Y%m')) ,'PeriodoT']= pd.to_datetime('202010',format='%Y%m')

In [27]:
comu_test['PeriodoT'].value_counts()

2020-10-01    230251
Name: PeriodoT, dtype: int64

In [28]:
comu_test['Tipo_comunicacion'].value_counts(dropna=False)

A    230251
Name: Tipo_comunicacion, dtype: int64

In [29]:
comu_test

,id,Id_Producto,Tipo,Producto-Tipo,Tipo_comunicacion,Fecha,Periodo,Lectura,PeriodoT
0,1,B,B,B-B,A,2020-08-07,2020-08-01,1,2020-10-01
1,1,B,B,B-B,A,2020-08-27,2020-08-01,1,2020-10-01
2,1,B,B,B-B,A,2020-09-30,2020-09-01,0,2020-10-01
3,1,B,B,B-B,A,2020-10-28,2020-10-01,1,2020-10-01
4,1,B,B,B-B,A,2020-10-15,2020-10-01,0,2020-10-01
...,...,...,...,...,...,...,...,...,...
230246,79537,B,B,B-B,A,2020-10-28,2020-10-01,0,2020-10-01
230247,79538,B,B,B-B,A,2020-10-28,2020-10-01,1,2020-10-01
230248,79539,C,D,C-D,A,2020-08-31,2020-08-01,0,2020-10-01
230249,79539,C,D,C-D,A,2020-09-02,2020-09-01,0,2020-10-01


In [30]:
comu_test_=comu_test.pivot_table( index=['id','PeriodoT','Producto-Tipo'],
                         values=['Lectura'],
                         aggfunc=[len,np.sum,np.mean,np.max,np.min])

In [31]:
comu_test_.reset_index(drop=False, inplace=True)

In [32]:
comu_test_

,id,PeriodoT,Producto-Tipo,len,sum,mean,amax,amin
,,,,Lectura,Lectura,Lectura,Lectura,Lectura
0,1,2020-10-01,B-B,5,3,0.6,1,0
1,2,2020-10-01,B-B,2,2,1.0,1,1
2,2,2020-10-01,C-D,2,2,1.0,1,1
3,3,2020-10-01,B-B,2,1,0.5,1,0
4,4,2020-10-01,C-D,2,1,0.5,1,0
...,...,...,...,...,...,...,...,...
87324,79537,2020-10-01,B-B,1,0,0.0,0,0
87325,79537,2020-10-01,C-D,2,0,0.0,0,0
87326,79538,2020-10-01,B-B,1,1,1.0,1,1


In [34]:
comu_test_.columns=['id','PeriodoT','Producto-Tipo','Count-comu','Lectura','LecturaMean','LecturaMax','LecturaMin']

In [35]:
comu_test_

,id,PeriodoT,Producto-Tipo,Count-comu,Lectura,LecturaMean,LecturaMax,LecturaMin
0,1,2020-10-01,B-B,5,3,0.6,1,0
1,2,2020-10-01,B-B,2,2,1.0,1,1
2,2,2020-10-01,C-D,2,2,1.0,1,1
3,3,2020-10-01,B-B,2,1,0.5,1,0
4,4,2020-10-01,C-D,2,1,0.5,1,0
...,...,...,...,...,...,...,...,...
87324,79537,2020-10-01,B-B,1,0,0.0,0,0
87325,79537,2020-10-01,C-D,2,0,0.0,0,0
87326,79538,2020-10-01,B-B,1,1,1.0,1,1
87327,79539,2020-10-01,B-B,1,1,1.0,1,1


# Hacemso el joint

In [36]:
comu_df =pd.concat([comu_ ,comu_test_ ], axis=0)

In [37]:
comu_df 

,id,PeriodoT,Producto-Tipo,Count-comu,Lectura,LecturaMean,LecturaMax,LecturaMin
0,1,2020-01-01,B-B,7,1,0.142857,1,0
1,1,2020-04-01,B-B,9,2,0.222222,1,0
2,1,2020-04-01,D-E,1,0,0.000000,0,0
3,1,2020-04-01,E-E,1,0,0.000000,0,0
4,1,2020-07-01,B-B,4,2,0.500000,1,0
...,...,...,...,...,...,...,...,...
87324,79537,2020-10-01,B-B,1,0,0.000000,0,0
87325,79537,2020-10-01,C-D,2,0,0.000000,0,0
87326,79538,2020-10-01,B-B,1,1,1.000000,1,1
87327,79539,2020-10-01,B-B,1,1,1.000000,1,1


In [38]:
comu_df['PeriodoT'].value_counts()

2020-04-01    237645
2020-01-01    100211
2019-07-01     99986
2020-07-01     93532
2020-10-01     87329
2019-10-01     78816
2019-04-01     16947
Name: PeriodoT, dtype: int64

In [39]:
comu_df.columns=['id','Periodo','Producto-Tipo','Count-comu','Lectura','LecturaMean','LecturaMax','LecturaMin']

In [40]:
comu_df

,id,Periodo,Producto-Tipo,Count-comu,Lectura,LecturaMean,LecturaMax,LecturaMin
0,1,2020-01-01,B-B,7,1,0.142857,1,0
1,1,2020-04-01,B-B,9,2,0.222222,1,0
2,1,2020-04-01,D-E,1,0,0.000000,0,0
3,1,2020-04-01,E-E,1,0,0.000000,0,0
4,1,2020-07-01,B-B,4,2,0.500000,1,0
...,...,...,...,...,...,...,...,...
87324,79537,2020-10-01,B-B,1,0,0.000000,0,0
87325,79537,2020-10-01,C-D,2,0,0.000000,0,0
87326,79538,2020-10-01,B-B,1,1,1.000000,1,1
87327,79539,2020-10-01,B-B,1,1,1.000000,1,1


# Producto Cartesiano

In [41]:
comu_df['Periodo'].unique()

array(['2020-01-01T00:00:00.000000000', '2020-04-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000', '2019-07-01T00:00:00.000000000',
       '2019-10-01T00:00:00.000000000', '2019-04-01T00:00:00.000000000',
       '2020-10-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [42]:
lista_fechas=['2019-04-01', 
'2019-07-01',
'2019-10-01', 
'2020-01-01',
'2020-04-01',
'2020-07-01',
'2020-10-01']

In [43]:
lista_clientes=comu_df['id'].unique().tolist()

In [44]:
len(lista_clientes)

77695

In [45]:
comu_df['Producto-Tipo'].value_counts(dropna=False)

B-B    374300
C-D    148429
D-E     67359
A-A     65097
E-E     59281
Name: Producto-Tipo, dtype: int64

In [46]:
lista_productos=['B-B',
'C-D',
'D-E',
'A-A',   
'E-E',]

In [47]:
len(lista_fechas), len(lista_clientes), len(lista_productos)

(7, 77695, 5)

In [48]:
cartesian_product = pd.MultiIndex.from_product([lista_clientes,lista_fechas, lista_productos ], 
                                               names = ['id','Periodo','Producto-Tipo'])
len(cartesian_product)

2719325

In [49]:
cartesian_product

MultiIndex([(    1, '2019-04-01', 'B-B'),
            (    1, '2019-04-01', 'C-D'),
            (    1, '2019-04-01', 'D-E'),
            (    1, '2019-04-01', 'A-A'),
            (    1, '2019-04-01', 'E-E'),
            (    1, '2019-07-01', 'B-B'),
            (    1, '2019-07-01', 'C-D'),
            (    1, '2019-07-01', 'D-E'),
            (    1, '2019-07-01', 'A-A'),
            (    1, '2019-07-01', 'E-E'),
            ...
            (77899, '2020-07-01', 'B-B'),
            (77899, '2020-07-01', 'C-D'),
            (77899, '2020-07-01', 'D-E'),
            (77899, '2020-07-01', 'A-A'),
            (77899, '2020-07-01', 'E-E'),
            (77899, '2020-10-01', 'B-B'),
            (77899, '2020-10-01', 'C-D'),
            (77899, '2020-10-01', 'D-E'),
            (77899, '2020-10-01', 'A-A'),
            (77899, '2020-10-01', 'E-E')],
           names=['id', 'Periodo', 'Producto-Tipo'], length=2719325)

In [50]:
full_df = pd.DataFrame(index = cartesian_product).reset_index()
full_df.tail()

,id,Periodo,Producto-Tipo
2719320,77899,2020-10-01,B-B
2719321,77899,2020-10-01,C-D
2719322,77899,2020-10-01,D-E
2719323,77899,2020-10-01,A-A
2719324,77899,2020-10-01,E-E


In [51]:
full_df['Periodo']=pd.to_datetime(full_df['Periodo'], format='%Y-%m-%d')

In [52]:
full_df.groupby(['id','Periodo']).agg({'Producto-Tipo':len})

Producto-Tipo
id    Periodo                  
1     2019-04-01              5
      2019-07-01              5
      2019-10-01              5
      2020-01-01              5
      2020-04-01              5
...                         ...
79539 2019-10-01              5
      2020-01-01              5
      2020-04-01              5
      2020-07-01              5
      2020-10-01              5

[543865 rows x 1 columns]

# Hacemos el Merge 

In [53]:
full_df = pd.merge(full_df,comu_df , on = ['id','Periodo','Producto-Tipo'], how = 'left')

In [54]:
full_df

,id,Periodo,Producto-Tipo,Count-comu,Lectura,LecturaMean,LecturaMax,LecturaMin
0,1,2019-04-01,B-B,NaN,NaN,NaN,NaN,NaN
1,1,2019-04-01,C-D,NaN,NaN,NaN,NaN,NaN
2,1,2019-04-01,D-E,NaN,NaN,NaN,NaN,NaN
3,1,2019-04-01,A-A,NaN,NaN,NaN,NaN,NaN
4,1,2019-04-01,E-E,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2719320,77899,2020-10-01,B-B,1.0,1.0,1.0,1.0,1.0
2719321,77899,2020-10-01,C-D,2.0,0.0,0.0,0.0,0.0
2719322,77899,2020-10-01,D-E,NaN,NaN,NaN,NaN,NaN
2719323,77899,2020-10-01,A-A,NaN,NaN,NaN,NaN,NaN


In [55]:
full_df.isnull().sum()

id                     0
Periodo                0
Producto-Tipo          0
Count-comu       2004859
Lectura          2004859
LecturaMean      2004859
LecturaMax       2004859
LecturaMin       2004859
dtype: int64

In [56]:
full_df.fillna(0,inplace=True)

In [57]:
# Hacemos un pickel en este punto
full_df.to_pickle('./3T/fulldf_base_comunicaciones.pkl',compression='zip')

## Sacamos LAG variables

In [58]:
full_df

,id,Periodo,Producto-Tipo,Count-comu,Lectura,LecturaMean,LecturaMax,LecturaMin
0,1,2019-04-01,B-B,0.0,0.0,0.0,0.0,0.0
1,1,2019-04-01,C-D,0.0,0.0,0.0,0.0,0.0
2,1,2019-04-01,D-E,0.0,0.0,0.0,0.0,0.0
3,1,2019-04-01,A-A,0.0,0.0,0.0,0.0,0.0
4,1,2019-04-01,E-E,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2719320,77899,2020-10-01,B-B,1.0,1.0,1.0,1.0,1.0
2719321,77899,2020-10-01,C-D,2.0,0.0,0.0,0.0,0.0
2719322,77899,2020-10-01,D-E,0.0,0.0,0.0,0.0,0.0
2719323,77899,2020-10-01,A-A,0.0,0.0,0.0,0.0,0.0


In [59]:
for y in ['Count-comu','Lectura','LecturaMean','LecturaMax','LecturaMin']:
    print('Trabajamos sobre: '+str(y))
    for x in [1,2,3,4,5]:
        print(x)
        full_df[str(y)+'-'+str(x)]=full_df.groupby(['id'])[y].shift(5*x)

Trabajamos sobre: Count-comu
1
2
3
4
5
Trabajamos sobre: Lectura
1
2
3
4
5
Trabajamos sobre: LecturaMean
1
2
3
4
5
Trabajamos sobre: LecturaMax
1
2
3
4
5
Trabajamos sobre: LecturaMin
1
2
3
4
5


In [60]:
lista_count=['id','Periodo','Producto-Tipo','Count-comu','Count-comu-1','Count-comu-2','Count-comu-3','Count-comu-4']
full_df[lista_count].head(50)

,id,Periodo,Producto-Tipo,Count-comu,Count-comu-1,Count-comu-2,Count-comu-3,Count-comu-4
0,1,2019-04-01,B-B,0.0,NaN,NaN,NaN,NaN
1,1,2019-04-01,C-D,0.0,NaN,NaN,NaN,NaN
2,1,2019-04-01,D-E,0.0,NaN,NaN,NaN,NaN
3,1,2019-04-01,A-A,0.0,NaN,NaN,NaN,NaN
4,1,2019-04-01,E-E,0.0,NaN,NaN,NaN,NaN
5,1,2019-07-01,B-B,0.0,0.0,NaN,NaN,NaN
6,1,2019-07-01,C-D,0.0,0.0,NaN,NaN,NaN
7,1,2019-07-01,D-E,0.0,0.0,NaN,NaN,NaN
8,1,2019-07-01,A-A,0.0,0.0,NaN,NaN,NaN
9,1,2019-07-01,E-E,0.0,0.0,NaN,NaN,NaN


In [61]:
full_df.isnull().sum()

id                     0
Periodo                0
Producto-Tipo          0
Count-comu             0
Lectura                0
LecturaMean            0
LecturaMax             0
LecturaMin             0
Count-comu-1      388475
Count-comu-2      776950
Count-comu-3     1165425
Count-comu-4     1553900
Count-comu-5     1942375
Lectura-1         388475
Lectura-2         776950
Lectura-3        1165425
Lectura-4        1553900
Lectura-5        1942375
LecturaMean-1     388475
LecturaMean-2     776950
LecturaMean-3    1165425
LecturaMean-4    1553900
LecturaMean-5    1942375
LecturaMax-1      388475
LecturaMax-2      776950
LecturaMax-3     1165425
LecturaMax-4     1553900
LecturaMax-5     1942375
LecturaMin-1      388475
LecturaMin-2      776950
LecturaMin-3     1165425
LecturaMin-4     1553900
LecturaMin-5     1942375
dtype: int64

In [62]:
full_df.fillna(-999, inplace=True)

In [63]:
full_df.isnull().sum()

id               0
Periodo          0
Producto-Tipo    0
Count-comu       0
Lectura          0
LecturaMean      0
LecturaMax       0
LecturaMin       0
Count-comu-1     0
Count-comu-2     0
Count-comu-3     0
Count-comu-4     0
Count-comu-5     0
Lectura-1        0
Lectura-2        0
Lectura-3        0
Lectura-4        0
Lectura-5        0
LecturaMean-1    0
LecturaMean-2    0
LecturaMean-3    0
LecturaMean-4    0
LecturaMean-5    0
LecturaMax-1     0
LecturaMax-2     0
LecturaMax-3     0
LecturaMax-4     0
LecturaMax-5     0
LecturaMin-1     0
LecturaMin-2     0
LecturaMin-3     0
LecturaMin-4     0
LecturaMin-5     0
dtype: int64

In [64]:
full_df.to_pickle('./3T/itau_comunicaciones_fe.pkl',compression='zip')